In [ ]:
!pip install emoji
import regex
import pandas as pd
import numpy as np
import emoji
import plotly.express as px
from collections import Counter
import matplotlib.pyplot as plt
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import re
import seaborn as sns
from datetime import *
import datetime as dt
from matplotlib.ticker import MaxNLocator
import emoji
from seaborn import *
from wordcloud import WordCloud , STOPWORDS , ImageColorGenerator
from nltk import *
from plotly import express as px
% matplotlib inline
def startsWithDateAndTime(s):
pattern = '^([0-9][0-9]+)(/)([0-9][0-9]+)(/)([0-9][0-9][0-9][0-9]),
(([0-9])|([0-9][0-9])+):([0-9][0-9]) (am|pm) -'
result = re.match(pattern, s)
if result:
return True
return False
def FindAuthor(s):
s=s.split(":")
if len(s)==2:
return True
else:
return False
def getDataPoint(line):
splitLine = line.split(' - ')
dateTime = splitLine[0]
date, time = dateTime.split(', ')

message = ' '.join(splitLine[1:])
if FindAuthor(message):
splitMessage = message.split(': ')
author = splitMessage[0]
message = ' '.join(splitMessage[1:])
else:
author = None
return date, time, author, message
from google.colab import drive
drive.mount('/content/gdrive')
parsedData = [] # List to keep track of data so it can be used by a Pandas
dataframe
conversation = '/content/gdrive/MyDrive/NLP/WhatsApp Chat with BE Project
Group 1 .txt'
with open(conversation, encoding="utf-8") as fp:
fp.readline() # Skipping first line of the file because contains
information related to something about end-to-end encryption
messageBuffer = []
date, time, author = None, None, None
while True:
line = fp.readline()
if not line:
break
line = line.strip()
if startsWithDateAndTime(line):
if len(messageBuffer) > 0:
parsedData.append([date, time, author, '

'.join(messageBuffer)])

messageBuffer.clear()
date, time, author, message = getDataPoint(line)
messageBuffer.append(message)
else:
messageBuffer.append(line)

df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author',
'Message']) # Initialising a pandas Dataframe.
df["Date"] = pd.to_datetime(df["Date"])
df.head(10)
df.Author.unique()
### Checking shape of dataset.
df.shape
### Checking basic information of dataset
df.info()
### Checking no. of null values in dataset
df.isnull().sum()
### Checking head part of dataset
df.head(50)
### Checking tail part of dataset
df.tail(50)
### Droping Nan values from dataset
df = df.dropna()
df = df.reset_index(drop=True)
df.shape
### Checking no. of authors of group
df['Author'].nunique()
### Checking authors of group
df['Author'].unique()

media_messages = df[df['Message'] == '<Media omitted>'].shape[0]
print(media_messages)
def split_count(text):
emoji_list = []
data = regex.findall(r'\X', text)
for word in data:
if any(char in emoji.UNICODE_EMOJI for char in word):
emoji_list.append(word)

return emoji_list
df["emoji"] = df["Message"].apply(split_count)
emojis = sum(df['emoji'].str.len())
print(emojis)
URLPATTERN = r'(https?://\S+)'

df['urlcount'] = df.Message.apply(lambda x: re.findall(URLPATTERN,
x)).str.len()
links = np.sum(df.urlcount)
print("Data science Community")
print("Media:",media_messages)
print("Emojis:",emojis)
print("Links:",links)

### Adding one more column of "Day" for better analysis, here we use
datetime library which help us to do this task easily.
weeks = {
0 : 'Monday',
1 : 'Tuesday',
2 : 'Wednesday',
3 : 'Thrusday',
4 : 'Friday',
5 : 'Saturday',
6 : 'Sunday'
}
df['Day'] = df['Date'].dt.weekday.map(weeks)

### Rearranging the columns for better understanding
df = df[['Date','Day','Time','Author','Message']]
### Changing the datatype of column "Day".
df['Day'] = df['Day'].astype('category')
### Looking newborn dataset.
df.head()
### Counting number of letters in each message
df['Letters'] = df['Message'].apply(lambda s : len(s))
### Counting number of word's in each message
df['Words'] = df['Message'].apply(lambda s : len(s.split(' ')))
### Function to count number of links in dataset, it will add extra column
and store information in it.
URLPATTERN = r'(https?://S+)'
df['Url_Count'] = df.Message.apply(lambda x: re.findall(URLPATTERN,
x)).str.len()
links = np.sum(df.Url_Count)
### Function to count number of media in chat.
MEDIAPATTERN = r'<Media omitted>'
df['Media_Count'] = df.Message.apply(lambda x : re.findall(MEDIAPATTERN,
x)).str.len()
media = np.sum(df.Media_Count)
### Looking updated dataset
df

df.tail(10)

total_messages = df.shape[0]
media_messages = df[df['Message'] == '<Media omitted>'].shape[0]
links = np.sum(df.Url_Count)
print('Group Chatting Stats : ')
print('Total Number of Messages : {}'.format(total_messages))
print('Total Number of Media Messages : {}'.format(media_messages))
print('Total Number of Links : {}'.format(links))

l = df.Author.unique()
for i in range(len(l)):
### Filtering out messages of particular user
req_df = df[df["Author"] == l[i]]
### req_df will contain messages of only one particular user
print(f'--> Stats of {l[i]} <-- ')
### shape will print number of rows which indirectly means the number of
messages
print('Total Message Sent : ', req_df.shape[0])
### Word_Count contains of total words in one message. Sum of all words/
Total Messages will yield words per message

words_per_message = (np.sum(req_df['Words']))/req_df.shape[0]
w_p_m = ("%.3f" % round(words_per_message, 2))
print('Average Words per Message : ', w_p_m)
### media conists of media messages
media = sum(req_df["Media_Count"])
print('Total Media Message Sent : ', media)
### links consist of total links
links = sum(req_df["Url_Count"])
print('Total Links Sent : ', links)
print()
print('----------------------------------------------------------\n')

### Word Cloud of mostly used word in our Group
text = " ".join(review for review in df.Message)
wordcloud = WordCloud(stopwords=STOPWORDS,
background_color="white").generate(text)
### Display the generated image:

plt.figure( figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### Creates a list of unique Authors
l = df.Author.unique()
for i in range(len(l)):
### Filtering out messages of particular user
req_df = df[df["Author"] == l[i]]
### req_df will contain messages of only one particular user
print(l[i],' -> ',req_df.shape[0])

l = df.Day.unique()
for i in range(len(l)):
### Filtering out messages of particular user
req_df = df[df["Day"] == l[i]]
### req_df will contain messages of only one particular user
print(l[i],' -> ',req_df.shape[0])

### Mostly Active Member in the Group
plt.figure(figsize=(9,6))
mostly_active = df['Author'].value_counts()
### Top 10 peoples that are mostly active in our Group is :
m_a = mostly_active.head(10)
bars = ['Kalpita Maam','Sneha Satish','Saloni Dhotre','Karan Doshi']
x_pos = np.arange(len(bars))
m_a.plot.bar()
plt.xlabel('Authors',fontdict={'fontsize': 14,'fontweight': 10})

plt.ylabel('No. of messages',fontdict={'fontsize': 14,'fontweight': 10})
plt.title('Mostly active member of Group',fontdict={'fontsize':
20,'fontweight': 8})
plt.xticks(x_pos, bars)
plt.show()

### Mostly Active day in the Group
plt.figure(figsize=(8,5))
active_day = df['Day'].value_counts()
### Top 10 peoples that are mostly active in our Group is :
a_d = active_day.head(10)
a_d.plot.bar()
plt.xlabel('Day',fontdict={'fontsize': 12,'fontweight': 10})
plt.ylabel('No. of messages',fontdict={'fontsize': 12,'fontweight': 10})
plt.title('Mostly active day of Week in the Group',fontdict={'fontsize':
18,'fontweight': 8})
plt.show()

### Top Media Contributor of Group
mm = df[df['Message'] == '<Media omitted>']
mm1 = mm['Author'].value_counts()
bars = ['Kalpita Maam','Sneha Satish','Saloni Dhotre','Karan Doshi']
x_pos = np.arange(len(bars))
top10 = mm1.head(10)
top10.plot.bar()
plt.xlabel('Authors',fontdict={'fontsize': 12,'fontweight': 10})
plt.ylabel('No. of media',fontdict={'fontsize': 12,'fontweight': 10})
plt.title('Top media contributor of Group',fontdict={'fontsize':
18,'fontweight': 8})
plt.xticks(x_pos, bars)
plt.show()

max_words = df[['Author','Words']].groupby('Author').sum()
m_w = max_words.sort_values('Words',ascending=False).head(10)
bars = ['Kalpita Maam','Sneha Satish','Saloni Dhotre','Karan Doshi']
x_pos = np.arange(len(bars))
m_w.plot.bar(rot=90)
plt.xlabel('Author')
plt.ylabel('No. of words')
plt.title('Analysis of members who has used max. no. of words in his/her
messages')
plt.xticks(x_pos, bars)
plt.show()

### Member who has shared max numbers of link in Group
max_words = df[['Author','Url_Count']].groupby('Author').sum()
m_w = max_words.sort_values('Url_Count',ascending=False).head(10)
bars = ['Kalpita Maam','Sneha Satish','Saloni Dhotre','Karan Doshi']
x_pos = np.arange(len(bars))
m_w.plot.bar(rot=90)
plt.xlabel('Author')
plt.ylabel('No. of links')
plt.title('Analysis of members who has shared max no. of links in Group')
plt.xticks(x_pos, bars)
plt.show()

### Time whenever our group is highly active
plt.figure(figsize=(8,5))
t = df['Time'].value_counts().head(20)
tx = t.plot.bar()
tx.yaxis.set_major_locator(MaxNLocator(integer=True)) #Converting y axis
data to integer
plt.xlabel('Time',fontdict={'fontsize': 12,'fontweight': 10})
plt.ylabel('No. of messages',fontdict={'fontsize': 12,'fontweight': 10})
plt.title('Analysis of time when Group was highly
active.',fontdict={'fontsize': 18,'fontweight': 8})
plt.show()

lst = []
for i in df['Time'] :
out_time = datetime.strftime(datetime.strptime(i,"%I:%M %p"),"%H:%M")
lst.append(out_time)
df['24H_Time'] = lst
df['Hours'] = df['24H_Time'].apply(lambda x : x.split(':')[0])
### Most suitable hour of day, whenever there will more chances of getting
response from group members.
plt.figure(figsize=(8,5))
std_time = df['Hours'].value_counts().head(15)
s_T = std_time.plot.bar()
s_T.yaxis.set_major_locator(MaxNLocator(integer=True)) #Converting y axis
data to integer
plt.xlabel('Hours (24-Hour)',fontdict={'fontsize': 12,'fontweight': 10})
plt.ylabel('No. of messages',fontdict={'fontsize': 12,'fontweight': 10})
plt.title('Most suitable hour of day.',fontdict={'fontsize':
18,'fontweight': 8})
plt.show()

active_m = df['Author'].unique()
for i in range(len(active_m)) :
# Filtering out messages of particular user
m_chat = df[df["Author"] == active_m[i]]
print(f'--- Author : {active_m[i]} --- ')
# Word Cloud of mostly used word in our Group
msg = ' '.join(x for x in m_chat.Message)
wordcloud = WordCloud(stopwords=STOPWORDS,
background_color="white").generate(msg)
plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()
print('___________________________________________________________________
_________________\n')

### Date on which our Group was highly active.
plt.figure(figsize=(8,5))
df['Date'].value_counts().head(15).plot.bar()
plt.xlabel('Date',fontdict={'fontsize': 14,'fontweight': 10})
plt.ylabel('No. of messages',fontdict={'fontsize': 14,'fontweight': 10})
plt.title('Analysis of Date on which Group was highly
active',fontdict={'fontsize': 18,'fontweight': 8})
plt.show()

z = df['Date'].value_counts()
z1 = z.to_dict() #converts to dictionary
df['Msg_count'] = df['Date'].map(z1)
### Timeseries plot
fig = px.line(x=df['Date'],y=df['Msg_count'])
fig.update_layout(title='Analysis of number of messages using TimeSeries
plot.',

xaxis_title='Month',
yaxis_title='No. of Messages')

fig.update_xaxes(nticks=20)
fig.show()

df['Year'] = df['Date'].dt.year
df['Mon'] = df['Date'].dt.month
months = {
1 : 'Jan',
2 : 'Feb',
3 : 'Mar',
4 : 'Apr',
5 : 'May',
6 : 'Jun',
7 : 'Jul',
8 : 'Aug',
9 : 'Sep',
10 : 'Oct',
11 : 'Nov',
12 : 'Dec'
}
df['Month'] = df['Mon'].map(months)
df.drop('Mon',axis=1,inplace=True)
### Mostly Active month
plt.figure(figsize=(12,6))
active_month = df['Month'].value_counts()
a_m = active_month
a_m.plot.bar()
plt.xlabel('Month',fontdict={'fontsize': 14,'fontweight': 10})
plt.ylabel('No. of messages',fontdict={'fontsize': 14,'fontweight': 10})
plt.title('Analysis of mostly active month.',fontdict={'fontsize': 20,

'fontweight': 8})

plt.show()

### Mostly Active month
plt.figure(figsize=(12,6))
active_month = df['Year'].value_counts()
a_m = active_month
a_m.plot.bar()
plt.xlabel('Month',fontdict={'fontsize': 14,'fontweight': 10})
plt.ylabel('No. of messages',fontdict={'fontsize': 14,'fontweight': 10})
plt.title('Analysis of mostly active month.',fontdict={'fontsize': 20,

'fontweight': 8})

plt.show()

z = df['Month'].value_counts()
z1 = z.to_dict() #converts to dictionary
df['Msg_count_monthly'] = df['Month'].map(z1)
plt.figure(figsize=(18,9))
sns.set_style("darkgrid")
sns.lineplot(data=df,x='Month',y='Msg_count_monthly',markers=True,marker='
o')
plt.xlabel('Month',fontdict={'fontsize': 14,'fontweight': 10})
plt.ylabel('No. of messages',fontdict={'fontsize': 14,'fontweight': 10})
plt.title('Analysis of mostly active month using line
plot.',fontdict={'fontsize': 20,'fontweight': 8})
plt.show()

z = df['Year'].value_counts()
z1 = z.to_dict() #converts to dictionary
df['Msg_count_monthly'] = df['Year'].map(z1)
plt.figure(figsize=(18,9))
sns.set_style("darkgrid")
sns.lineplot(data=df,x='Year',y='Msg_count_monthly',markers=True,marker='o
')
plt.xlabel('Month',fontdict={'fontsize': 14,'fontweight': 10})
plt.ylabel('No. of messages',fontdict={'fontsize': 14,'fontweight': 10})
plt.title('Analysis of mostly active month using line
plot.',fontdict={'fontsize': 20,'fontweight': 8})
plt.show()

### Total message per year
### As we analyse that the group was created in mid 2019, thats why number
of messages in 2019 is less.
plt.figure(figsize=(12,6))
active_month = df['Year'].value_counts()
a_m = active_month
a_m.plot.bar()
plt.xlabel('Year',fontdict={'fontsize': 14,'fontweight': 10})
plt.ylabel('No. of messages',fontdict={'fontsize': 14,'fontweight': 10})
plt.title('Analysis of mostly active year.',fontdict={'fontsize':
20,'fontweight': 8})
plt.show()

df2 = df.groupby(['Hours', 'Day'], as_index=False)["Message"].count()
df2 = df2.dropna()
df2.reset_index(drop = True,inplace = True)
### Analysing on which time group is mostly active based on hours and day.
analysis_2_df = df.groupby(['Hours', 'Day'],
as_index=False)["Message"].count()
### Droping null values
analysis_2_df.dropna(inplace=True)
analysis_2_df.sort_values(by=['Message'],ascending=False)

day_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thrusday', 'Friday',
'Saturday', 'Sunday']
plt.figure(figsize=(15,8))

sns.heatmap(
df2,
x=analysis_2_df['Hours'],
y=analysis_2_df['Day'],
size_scale = 500,
size = analysis_2_df['Message'],
y_order = day_of_week[::-1],
color = analysis_2_df['Message'],
palette = sns.cubehelix_palette(128)
)
plt.show()